# Visualization Control System Basic Tutorial<a id='top' class="tocSkip"> </a>

VCS allows scientists to produce highly customized plots. Everything can be precisely and logically controlled, without engaging in a guessing game.


The CDAT software was developed by LLNL. This tutorial was written by Charles Doutriaux. This work was performed under the auspices of the U.S. Department of Energy by Lawrence Livermore National Laboratory under Contract DE-AC52-07NA27344.

This tutorial is a high level *flight* of VCS capabilities.

We encourage you to also consult the [VCS Principles Notebook](../VCS_Principles/VCS_Principles.ipynb) and the [CDMS 101](../../cdms/cdms2_101/cdms2_101.ipynb) notebook.

[Download the Jupyter Notebook](VCS_Basics.ipynb)

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Setup-Notebook-" data-toc-modified-id="Setup-Notebook--1">Setup Notebook <a id="setup"></a></a></span></li><li><span><a href="#Opening-Data-Files-" data-toc-modified-id="Opening-Data-Files--2">Opening Data Files <a id="open"></a></a></span></li><li><span><a href="#Querying-File-" data-toc-modified-id="Querying-File--3">Querying File <a id="query"></a></a></span></li><li><span><a href="#First-Plot-" data-toc-modified-id="First-Plot--4">First Plot <a id="first_plot"></a></a></span></li><li><span><a href="#Subsetting-Data" data-toc-modified-id="Subsetting-Data-5">Subsetting Data<a id="subset"></a></a></span></li><li><span><a href="#Using-Isolines" data-toc-modified-id="Using-Isolines-6">Using Isolines<a id="isolines"></a></a></span></li><li><span><a href="#Cross-Sections" data-toc-modified-id="Cross-Sections-7">Cross Sections<a id="cross"></a></a></span></li><li><span><a href="#Easy-Plot-Tweaks" data-toc-modified-id="Easy-Plot-Tweaks-8">Easy Plot Tweaks<a id="tweaks"></a></a></span></li><li><span><a href="#Overlays" data-toc-modified-id="Overlays-9">Overlays<a id="overlay"></a></a></span></li><li><span><a href="#Manipulating-Variables" data-toc-modified-id="Manipulating-Variables-10">Manipulating Variables<a id="manipulate"></a></a></span></li><li><span><a href="#Averaging-" data-toc-modified-id="Averaging--11">Averaging <a id="average"></a></a></span><ul class="toc-item"><li><span><a href="#Average-Across-Time" data-toc-modified-id="Average-Across-Time-11.1">Average Across Time</a></span></li><li><span><a href="#Remove-Zonal-Mean" data-toc-modified-id="Remove-Zonal-Mean-11.2">Remove Zonal Mean</a></span></li><li><span><a href="#Time-Difference" data-toc-modified-id="Time-Difference-11.3">Time Difference</a></span></li><li><span><a href="#Time-Offset" data-toc-modified-id="Time-Offset-11.4">Time Offset</a></span></li></ul></li><li><span><a href="#Other-graphic-methods" data-toc-modified-id="Other-graphic-methods-12">Other graphic methods<a id="others"></a></a></span><ul class="toc-item"><li><span><a href="#Isofill" data-toc-modified-id="Isofill-12.1">Isofill<a id="isofill"></a></a></span></li><li><span><a href="#Vectors-" data-toc-modified-id="Vectors--12.2">Vectors <a id="vectors"></a></a></span></li><li><span><a href="#Streamlines-" data-toc-modified-id="Streamlines--12.3">Streamlines <a id="stream"></a></a></span></li></ul></li><li><span><a href="#Meshfill-" data-toc-modified-id="Meshfill--13">Meshfill <a id="meshfill"></a></a></span></li><li><span><a href="#Projections-" data-toc-modified-id="Projections--14">Projections <a id="proj"></a></a></span></li><li><span><a href="#Going-Deeper" data-toc-modified-id="Going-Deeper-15">Going Deeper<a id="deeper"></a></a></span></li></ul></div>

# Setup Notebook <a id="setup"></a> 

<a href="#toc">Back To Top</a>

First let's download some sample data, specifically the three NetCDF files: ta_ncep_87-6-88-4.nc, clt.nc, and sampleCurveGrid4.nc, which will be stored in the same directory as this notebook.

In [ ]:
from __future__ import print_function
import requests
import os

for filename in ['ta_ncep_87-6-88-4.nc', 'clt.nc', 'sampleCurveGrid4.nc']:
    if not os.path.exists(filename):
        r = requests.get("https://cdat.llnl.gov/cdat/sample_data/{}".format(filename), stream=True)
        with open(filename,"wb") as f:
            for chunk in r.iter_content(chunk_size=1024):
                if chunk:
                    f.write(chunk) # write this chunk to a local version of the file


# Opening Data Files <a id="open"></a>
<a href="#toc">Back To Top</a>

First we will open one of our demo files, which is done via the `cdms2` module.

In [ ]:
import cdms2

f = cdms2.open("ta_ncep_87-6-88-4.nc")

# Querying File <a id="query"></a>
<a href="#toc">Back To Top</a>

Next we will query the file to learn which variables it contains.

In [ ]:
f.variables.keys()

We can obtain information on a specific variable. In the next line of code we look at the `ta` variable which is the Air Temperature in degrees Kelvin (K). The file contains 11 months of of Air Temperature data (the "T" axis), with 17 different atmospheric pressure levels (the Z axis), and spans the whole globe with latitude ranging from 90 to -90 and longitude from 0 to 360. The latitude axis is Y and the longitude axis is X. (If this seems backwards, think of the values of latitude increasing or decreasing along the Y axis though the actual lines of latitude - or parallels - follow the X axis).

In [ ]:
f["ta"].info()

In general, with the latitude dimension having a length of 73 (meaning there are 73 temperature values from -90 to 90 degrees) and the longitude dimension having a length of 144 (or 144 temperature values from 0 to 360 degrees), a single temperature value spans a 2.5 x 2.5 degree portion of the globe. The exception is in the latitude dimension which would normally have a length of 72 if all sections of the grid were 2.5 degrees. Instead it has a length of 73 because the first and last grid elements (at -90 and 90 degrees) are 1.25 degrees tall  each not 2.5. You can see this by displaying the latitude slices in the data using the `.getBounds()` method.

In [ ]:
f["ta"].getLatitude().getBounds()

# First Plot <a id="first_plot"></a>

<a href="#toc">Back To Top</a>

Let's visualize the `ta` variable. For this we will need the `vcs` module.
We will also need a vcs `canvas` upon which to plot the data.

In [ ]:
import vcs
canvas = vcs.init()
canvas.plot(f["ta"]) # This plots the first time interval (1987/6/1) and the first atmospheric level (1000).

# Subsetting Data<a id="subset"></a>

<a href="#toc">Back To Top</a>


In the above plot we used all the variable dimensions (time, level, latitude and longitude) and therefore got a ***boxfill***. (If the data has at least two dimensions for the variable being plotted, the default plot type is boxfill.) 

You can load and use a subset of the dimensions. Notice how we can use either the actual dimension value or indices (via `slice`). For this dataset the longitude values range from 0 to 360, so specifying a -90 value for longintude means 360-90 = 270.

In [ ]:
data = f["ta"](longitude=-90, latitude=40, level=500, time=slice(0,1))
print(data)

In the previous example we fixed all dimensions and therefore got a single value. Let's subset longitude to a range rather than a single value and plot it.

The "squeeze" parameter in the first line below prevents dimensions of length 1 (e.g. latitude, level, and time) from being plotted.

In [ ]:
data = f["ta"](longitude=(0,180), latitude=40, level=500, time=slice(0,1), squeeze=1)
canvas.clear()
canvas.plot(data)

# Using Isolines<a id="isolines"></a>

<a href="#toc">Back To Top</a>

Now if we want an ***isoline*** rather than the default ***boxfill***, we will need to create the associated `isoline` graphic rendering object using vcs' `createisoline` method.  

In [ ]:
isoline = vcs.createisoline()
data = f("ta")
canvas.clear()
canvas.plot(data, isoline)

# Cross Sections<a id="cross"></a>

<a href="#toc">Back To Top</a>

Let's plot a cross section of the data.

In [ ]:
canvas.clear()
data = f["ta"](longitude=-90, latitude=(-90,90), level=(1000,100), time=slice(0,1), squeeze=1)
canvas.plot(data, isoline)

We can also colorize the isoline if we want and turn on labels.

In [ ]:
levels = vcs.mkscale(*vcs.minmax(data))
colors = vcs.getcolors(levels)
isoline.levels = levels
isoline.linecolors = colors
isoline.label = True
isoline.textcolors= colors
canvas.clear()
canvas.plot(data, isoline)

# Easy Plot Tweaks<a id="tweaks"></a>

<a href="#toc">Back To Top</a>


We can also control the dimensions' "range" on the plot via the `isoline` graphic rendering object,

In [ ]:
data = f["ta"](longitude=-90, time=slice(0,1), squeeze=1)
# The next two lines put the North Pole on the left of the plot
isoline.datawc_x1 = 90
isoline.datawc_x2 = -90
isoline.datawc_y1 = 3  # log10(1000)
isoline.datawc_y2 = 2.3  # log10(200)

Or tweak the axes' scale (turn the y axis - the atmospheric pressure levels - into a base 10 logarithmic scale), 

In [ ]:
isoline.yaxisconvert = "log10"

Or change the default labels.

In [ ]:
isoline.yticlabels1 = {1000:"1000", 800:"800", 500: "500", 20:"20"}
canvas.clear()
canvas.plot(data, isoline)

# Overlays<a id="overlay"></a>

<a href="#toc">Back To Top</a>


Now let's overlay another variable. Since it has different values, we will create a separate `isoline` graphic rendering object, `iso2`.

In [ ]:
f2 = cdms2.open("clt.nc")
u = f2("u", longitude=-90., time=slice(0,1), squeeze=1)  # First time slice
iso2 = vcs.createisoline(source=isoline)  # Essentially makes a copy
levels = vcs.mkscale(*vcs.minmax(u))
iso2.levels = levels
colors = ["blue",]  # Blue lines
iso2.linecolors = colors
iso2.textcolors = colors
canvas.clear()
canvas.plot(data, isoline)
canvas.plot(u, iso2)

# Manipulating Variables<a id="manipulate"></a>

<a href="#toc">Back To Top</a>

Manipulating variables is easy. Let's plot the temperature in Fahrenheit rather than Kelvin.

As the `.info()` command shows, even though we've changed the values of the `ta` variable to be in Fahrenheit, the title still says the Air Temperature is in Kelvin, the units are not specified, and everything else remains the same as before the conversion into Fahrenheit.

In [ ]:
ta_f = (f["ta"]-273.15)*9/5+32
ta_f.info()

The next two lines specify that the units are Fahrenheit (for plotting purposes) and adjusts the title to match the data. `$^o$F` is the sequence of characters that is needed to properly plot the degree symbol and "F" on the image ($^o$F). When we type `ta_f.info()`, we can see that the units and title have been updated.

In [ ]:
ta_f.units = r"$^o$F"
ta_f.title = r"Air Temperature ($^o$F)"
ta_f.info()

In [ ]:
canvas.clear()
canvas.plot(ta_f)

We can also combine variables together or perform other calculations.

In [ ]:
u = f2("u")
v = f2("v")
speed = cdms2.MV2.sqrt(u**2 + v**2)
speed.id = "Wind Speed"
speed.units = "m/s"
canvas.clear()
canvas.plot(speed)

# Averaging <a id="average"></a>

<a href="#toc">Back To Top</a>

## Average Across Time
Using the genutil module we can also easily average across one of the dimensions, such as `time` in the example below.

In [ ]:
import genutil, cdutil
data = f("ta", time=slice(0,5))
ta_avg = genutil.averager(data, axis='t')
canvas.clear()
canvas.plot(ta_avg)

## Remove Zonal Mean
We can also remove the zonal mean by averaging over longitude. For each latitude slice in the data, the `averager` function below averages the tempeartures over all the longitudes and stores a single averaged value for that latitude. Hence, after averaging `ta_zm` no longer has longitude values.

In [ ]:
ta = f("ta")
ta_zm = genutil.averager(f("ta"), axis="x")
ta_zm.shape

To remove the zonal mean we need to subtract `ta_zm` from `ta`, but to do that the two need to be the same size and shape (11, 17, 73, 144). Hence we need to grow `ta_zm` in the longitudinal direction. This will take the average temperature for each latitude band and spread it across all 144 longitude values in the file to match `ta`.

In [ ]:
ta, ta_zm = genutil.grower(ta, ta_zm)
ta_zm.shape

In [ ]:
canvas.clear()
canvas.plot(ta-ta_zm) # the zonal mean has been removed.

## Time Difference
We can do a simple time difference (last - first time point).

In [ ]:
diff = ta[-1] - ta[0]
diff.shape

## Time Offset
We can offset all points by one day.

In [ ]:
offset = ta[1:] - ta[:-1]
offset.shape

Note that the other axes are preserved.

In [ ]:
offset.getAxisList()

# Other graphic methods<a id="others"></a>

Now we will move on to the topic of controlling the graphics output. So far, we have usually allowed VCS to chose the defaults.

## Isofill<a id="isofill"></a>

<a href="#toc">Back To Top</a>

We have already seen the default `boxfill` and how to use `isoline`. Another available plot type combines these two and is called `isolfill`. In the example below the isofill object is called `gm` (for graphic method) and is generated using the `createisofill` method. The plot covers North America, Greenland and the north Atlantic at the 500 millibar level and the first time value of 1987/6/1 or June 1, 1987.

In [ ]:
ta = f("ta", longitude=(-180, 0), latitude=(0,90), level=500, squeeze=1)
gm = vcs.createisofill()
canvas.clear()
canvas.plot(ta, gm)

## Vectors <a id="vectors"></a>

<a href="#toc">Back To Top</a>

The `vector` plot type requires 2 variables. Here they are the u and v components of the wind. The `vector` graphic rendering object (`gm`) is generated via the `createvector` method. 

In [ ]:
u = f2("u", longitude=(-180, 0), latitude=(0,90), squeeze=1)
v = f2("v", longitude=(-180, 0), latitude=(0,90), squeeze=1)
gm = vcs.createvector()
canvas.clear()
canvas.plot(u, v, gm)

## Streamlines <a id="stream"></a>

<a href="#toc">Back To Top</a>

The `streamlines` plot type also requires 2 variables. Here we are using the u and v components of the wind, which we defined above.

In [ ]:
gm = vcs.createstreamline()
canvas.clear()
canvas.plot(u,v, gm)

# Meshfill <a id='meshfill'></a>

<a href="#toc">Back to Top</a>

If your NetCDF file is CF compliant, VCS can plot generic grids right out of the box. Here we are using a sample curvilinear grid. The sample variable ("sample") is assigned to the variable name `curv` and `m` is the meshfill graphic rendering object created via the `createmeshfill` method. 

In [ ]:
f3 = cdms2.open("sampleCurveGrid4.nc")
curv = f3("sample")

m = vcs.createmeshfill()

The first plot we will look at is the raw data that makes up the curviliniear grid. The grid itself is 32 rows by 48 columns.

The sample data that fills this grid were created to have some data to plot. The data values were generated by starting with a low value of 0.0 in the 0,0 location and increasing to a maximum value of 1535.0 in the last row and column. If we plot this data using 32 rows and 48 columns we get the following image which has the highest value of 1535.0 in the upper right-hand corner.

In [ ]:
canvas.clear()
canvas.plot(curv.filled())

The purpose of this curvilinear grid is for modeling the oceans so this grid is wrapped around the Earth in such a way that the North Pole is over land rather than over the Arctic Ocean. Hence in the plot below the "North Pole" of the grid - the white area where there is no data -  is over land, i.e. northern Canada.

In [ ]:
canvas.clear()
canvas.plot(curv, m)

# Projections <a id="proj"></a>

<a href="#toc">Back To Top</a>

You can also adjust the projection. First create a projection graphic rendering object (`proj`) by using the `createprojection` method. Specify the projection type and the standard parallels, then assign that projection to the `.projection` attribute of your object. Here we've created a boxfill object called `gm` and assigned it the Lambert project with the first standard parallel at 0 and the second standard parallel at 89 degrees. We are using the `ta` object from the <a href="#isofill">Isofill</a> example.

In [ ]:
proj = vcs.createprojection()
proj.type = "lambert"
proj.standardparallel1 = 0
proj.standardparallel2 = 89
gm = vcs.createboxfill()
gm.projection = proj
canvas.clear()
canvas.plot(ta, gm)

# Going Deeper<a id="deeper"></a>

<a href="#toc">Back To Top</a>


The [Tutorials](https://cdat.llnl.gov/tutorials) page offers in-depth details for each of the graphic methods.